In [1]:
from langchain_groq import ChatGroq
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

In [3]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=api_key
)

In [4]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
import  re
loader = WebBaseLoader("https://careers.nike.com/store-manager-head-coach-dundrum/job/R-63740")
page_data = loader.load().pop().page_content
lines = page_data.split('\n')
clean_lines = [re.sub(r'\s+', ' ', line).strip() for line in lines if line.strip()]
clean_data = '\n'.join(clean_lines)
page_data = clean_data
print(page_data)

Store Manager (Head Coach) - Dundrum
Skip to main content
Open Virtual Assistant
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Language
Select a Language
Deutsch
English
Español (España)
Español (América Latina)
Français
Italiano
Nederlands
Polski
Tiếng Việt
Türkçe
简体中文
繁體中文
עִברִית
한국어
日本語
Careers
Close Menu
Careers
Chat
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Jordan Careers
Converse Careers
Language
Menu
Return to Previous Menu
Select a Language
Deutsch
English
Español (España)
Español (América Latina)
Français
Italiano
Nederlands
Polski
Tiếng Việt
Türkçe
简体中文
繁體中文
עִברִית
한국어
日本語
Back to Search
Store Manager (Head Coach) - Dundrum
Categories ID
Categories URL
Position Type
Full Time
Date Posted
Primary Quest ID
Second Quest ID
Job Classification
Career area
Stores
Location
16 Sandyford Road, Dublin Dundrum, Dundrum, Dublin D16 E0E8, Ireland
Job ID
R-63740
Apply Now
Share Job
Share Job Posting
Facebook
Opens In A New Tab
LinkedIn
Link Opens In New Window
Email
Close-Med

In [6]:
from langchain_core.prompts import PromptTemplate

In [7]:
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)


In [8]:
chain_extract = prompt_extract | llm

In [9]:
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

Here is the extracted job posting in JSON format:

```
{
  "role": "Store Manager (Head Coach)",
  "experience": "Demonstrated store management experience",
  "skills": [
    "Ability to lead the delivery of a high level of customer service",
    "Ability to build, lead with courage and manage high performing teams",
    "Effective communication skills",
    "Fluency in English and local language",
    "Ability to use retail business systems & Microsoft Office"
  ],
  "description": "As a Nike Store Manager (Head Coach), your mission is to bring Nike to life for our consumers. The Head Coach is accountable for developing both leaders and athletes to successfully implement Nike’s programs and delivering premium consumer experience in service of driving positive business results."
}
```


In [10]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Store Manager (Head Coach)',
 'experience': 'Demonstrated store management experience',
 'skills': ['Ability to lead the delivery of a high level of customer service',
  'Ability to build, lead with courage and manage high performing teams',
  'Effective communication skills',
  'Fluency in English and local language',
  'Ability to use retail business systems & Microsoft Office'],
 'description': 'As a Nike Store Manager (Head Coach), your mission is to bring Nike to life for our consumers. The Head Coach is accountable for developing both leaders and athletes to successfully implement Nike’s programs and delivering premium consumer experience in service of driving positive business results.'}

In [11]:
type(json_res)

dict

In [12]:
import pandas as pd
df = pd.read_csv("dummy_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [13]:
import  chromadb
import uuid

In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
job = json_res
job['skills']

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)